# Классификация диалектизмов архангельских говоров

В этом ноутбуке представлена модель для классификации диалектизмов, обученная на материале архангельских говоров.

Продемонстрируем, как использовать эту модель для классификации новых слов.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [3]:
# в словаре используются следующие символы
vocab = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
vocab_to_int = {word: i + 1 for i, word in enumerate(vocab)}

# паддинговый символ
vocab_to_int['PAD'] = 0

vocab_size = len(vocab_to_int.keys())

# метки классов
dec = ['Фонетическая',
       'Словообразовательная',
       'Лексическая',
       'Грамматическая']


def encode(text, max_char_length=20):
    '''
    Преобразует текст в формат One-Hot Encoding (OHE).

    Параметры:
    text (str): Входной текст, который нужно преобразовать.
    max_char_length (int): Максимальная длина текста для преобразования.
                           По умолчанию 20.

    Возвращает:
    np.ndarray: Массив размером [max_char_length, vocab_size],
                где каждый символ текста преобразован в OHE-вектор.
    '''

    encoded = np.zeros([max_char_length, vocab_size], dtype=int)
    i = 0
    for letter in text:
        if i >= max_char_length:
            break
        if letter.lower() in vocab_to_int:
            encoded[i][vocab_to_int[letter.lower()]] = 1
        i += 1
    return encoded

In [21]:
def encode_word(word, max_char_length=20):
    '''
    Преобразует слово для подачи на вход нейронной сети.

    Параметры:
    word (str): Входное слово, которое нужно преобразовать.
    max_char_length (int): Максимальная длина слова для преобразования.
                           По умолчанию 20.

    Возвращает:
    np.ndarray: Массив размером [max_char_length, vocab_size],
                где каждый символ текста преобразован в OHE-вектор.
    '''

    word = encode(word.lower().replace('ё', 'е'))
    word = word.astype('float32')
    word = word.reshape(1, max_char_length, vocab_size)
    return word

Для примера классификации использованы следующие слова:

*   "наблюдник" - словообразовательная категория,
*   "рукотерник" - словообразовательная категория,
*   "улеч" - фонетическая категория,
*   "итти" - фонетическая категория,
*   "складь" - грамматическая категория,
*   "бывата" - грамматическая категория,
*   "катанцы" - лексическая категория,
*   "халупа" - лексическая категория.





In [25]:
words = ['наблюдник',
         'рукотерник',
         'жырнов',
         'улец',
         'складь',
         'бывата',
         'катанцы',
         'халупа']
test_data = {w: encode_word(w) for w in words}

In [31]:
model = load_model('cnn_uni.keras')
for word, encoded_word in test_data.items():
    prediction = model.predict(encoded_word)
    print(f'{word} принадлежит к категории:' \
          f'{np.argmax(prediction), dec[np.argmax(prediction)]}.')
    print(f'Вероятность по всем классам: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
наблюдник принадлежит к категории:(1, 'Словообразовательная').
Вероятность по всем классам: [[0.04099802 0.7292632  0.20197457 0.02776427]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
рукотерник принадлежит к категории:(1, 'Словообразовательная').
Вероятность по всем классам: [[0.05156646 0.83593243 0.08977945 0.02272156]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
жырнов принадлежит к категории:(0, 'Фонетическая').
Вероятность по всем классам: [[0.38351968 0.12443283 0.20450273 0.2875447 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
улец принадлежит к категории:(0, 'Фонетическая').
Вероятность по всем классам: [[0.36255327 0.12864909 0.19026917 0.3185284 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
складь принадлежит к категории:(3, 'Грамматическая').
Вероятность по всем классам: [[0.3108922  0.10131159 0.1467913  0.4410049 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
бывата принадлежит к категории:(2, 'Лексическая').
Вероятность по всем классам: [[0.26233214 0.11366124 